# 🔬 Predicción de Diabetes con Machine Learning
Trabajo Práctico Integrador

En este notebook se realiza un análisis de datos, entrenamiento de modelos supervisados y creación de una app gráfica para la predicción de diabetes.

## 1. Carga y exploración de datos

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset
df = pd.read_csv("/content/diabetes.csv")
df.head()


In [ ]:

# Análisis de tipos de datos y valores faltantes
print(df.info())
print(df.describe())

# Detección de ceros sospechosos
cols_con_ceros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
print(df[cols_con_ceros].isin([0]).sum())


In [ ]:

# Visualizaciones básicas
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de Correlación")
plt.show()

sns.countplot(data=df, x="Outcome")
plt.title("Distribución de Pacientes con/sin Diabetes")
plt.show()


## 2. Preprocesamiento de datos

In [ ]:

# Reemplazo de ceros por la mediana
for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    mediana = df[col].median()
    df[col] = df[col].replace(0, mediana)

print(df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].isin([0]).sum())


In [ ]:

# Normalización
from sklearn.preprocessing import StandardScaler

X = df.drop("Outcome", axis=1)
y = df["Outcome"]

scaler = StandardScaler()
X_normalizado = scaler.fit_transform(X)


In [ ]:

# División en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalizado, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")


## 3. Entrenamiento de Modelos

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Regresión Logística
modelo_log = LogisticRegression()
modelo_log.fit(X_train, y_train)
y_pred_log = modelo_log.predict(X_test)
accuracy_log = accuracy_score(y_test, y_pred_log)
print(f"Precisión de Regresión Logística: {accuracy_log:.2f}")

# Árbol de Decisión
modelo_tree = DecisionTreeClassifier(random_state=42)
modelo_tree.fit(X_train, y_train)
y_pred_tree = modelo_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(f"Precisión del Árbol de Decisión: {accuracy_tree:.2f}")

# Random Forest
modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Precisión del Bosque Aleatorio: {accuracy_rf:.2f}")


## 4. Evaluación y comparación

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report

modelos = [
    ("Regresión Logística", modelo_log, y_pred_log),
    ("Árbol de Decisión", modelo_tree, y_pred_tree),
    ("Random Forest", modelo_rf, y_pred_rf),
]

for nombre, modelo, y_pred in modelos:
    print(f"\nModelo: {nombre}")
    print(f"Score: {modelo.score(X_test, y_test):.2f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("Matriz de Confusión:")
    print(confusion_matrix(y_test, y_pred))
    print("Reporte de Clasificación:")
    print(classification_report(y_test, y_pred))


## 5. Exportación del modelo

In [ ]:

import joblib

# Exportar el mejor modelo (Random Forest)
joblib.dump(modelo_rf, "modelo_diabetes.pkl")

# Descargar el archivo desde Colab
from google.colab import files
files.download("modelo_diabetes.pkl")
